In [1]:
!pip install azure-cognitiveservices-vision-face

You should consider upgrading via the 'C:\Users\Anahi\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
cog_key = '7792715916944474bc677dda15495c43'
cog_endpoint = 'https://ejemplo-caras.cognitiveservices.azure.com/'
print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

Ready to use cognitive services at https://ejemplo-caras.cognitiveservices.azure.com/ using key 7792715916944474bc677dda15495c43


In [3]:
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
import faces
import os
%matplotlib inline

In [12]:
# Crear un cliente de detección facial.
face_client = FaceClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

In [ ]:

# Abrir una imagen
image_path ='../LabCV/Data/vision/store_cam2.jpg'
image_stream = open(image_path, "rb")

# Detectar caras
detected_faces = face_client.face.detect_with_stream(image=image_stream)

# Mostrar las caras (código en python_code/faces.py)
faces.show_faces(image_path, detected_faces)

In [ ]:
#Ejecute la siguiente celda para ver los ID de la cara de algunos clientes.
# Abrir una imagen

image_path ='../LabCV/Data/vision/store_cam3.jpg'
image_stream = open(image_path, "rb")

# Detectar caras
detected_faces = face_client.face.detect_with_stream(image=image_stream)

# Mostrar las caras (código en python_code/faces.py)
faces.show_faces(image_path, detected_faces, show_id=True)

Analizar los atributos faciales
Face puede ir mucho más allá del simple reconocimiento facial. 
 También puede analizar las características y expresiones de las caras 
 para indicar la edad y el estado emocional. 
 Ejecute el siguiente código para analizar los atributos faciales de un cliente.

In [ ]:
# Abrir una imagen
image_path ='../LabCV/data/vision/store_cam1.jpg'
image_stream = open(image_path, "rb")

# Detectar caras y determinados atributos faciales
attributes = ['age', 'emotion']
detected_faces = face_client.face.detect_with_stream(image=image_stream, return_face_attributes=attributes)

# Mostrar las caras y los atributos (código en python_code/faces.py)
faces.show_face_attributes(image_path, detected_faces)

Búsqueda de caras similares
Los ID de caras creados para cada cara detectada se usan para identificar cada una de ellas. Puede usar estos ID para comparar una cara detectada con otras detectadas anteriormente y encontrar caras con características similares.

Por ejemplo, ejecute la siguiente celda para comparar el cliente de una imagen con los clientes de otra y encontrar una cara que coincida.

In [ ]:
# Obtener el identificador de la primera cara de la imagen 1
image_1_path ='../LabCV/data/vision/store_cam3.jpg'
image_1_stream = open(image_1_path, "rb")
image_1_faces = face_client.face.detect_with_stream(image=image_1_stream)
face_1 = image_1_faces[0]

# Obtener los identificadores de caras en una segunda imagen
image_2_path ='../LabCV/data/vision/store_cam2.jpg'
image_2_stream = open(image_2_path, "rb")
image_2_faces = face_client.face.detect_with_stream(image=image_2_stream)
image_2_face_ids = list(map(lambda face: face.face_id, image_2_faces))

# Encontrar las caras de la imagen 2 que sean similares a la de la imagen 1
similar_faces = face_client.face.find_similar(face_id=face_1.face_id, face_ids=image_2_face_ids)

# Mostrar la cara de la imagen 1 y las caras similares de la imagen 2 (código en python_code/face.py)
faces.show_similar_faces(image_1_path, face_1, image_2_path, image_2_faces, similar_faces)

Reconocimiento de caras
Hasta ahora, hemos visto que Face puede detectar caras y características faciales e identificar dos caras similares entre sí. Podemos ir más allá si implementamos una solución de reconocimiento facial en la que se entrene a Face para reconocer la cara de una persona en concreto. Esto puede ser útil en diferentes casos, como para etiquetar fotografías de amigos automáticamente en redes sociales o usar el reconocimiento facial como un sistema biométrico de verificación de identidad.

Para ver cómo funciona, supongamos que Northwind Traders quiere usar el reconocimiento facial para garantizar que solo los empleados autorizados del departamento de TI pueda acceder a los sistemas seguros.

Lo primero que haremos es crear un grupo de personas que represente a los empleados autorizados.

In [ ]:
group_id = 'employee_group_id'
try:
    # Elimine el grupo si ya existe
    face_client.person_group.delete(group_id)
except Exception as ex:
    print(ex.message)
finally:
    face_client.person_group.create(group_id, 'employees')
    print ('Group created!')
    group_id = 'employee_group_id'
try:
    # Elimine el grupo si ya existe
    face_client.person_group.delete(group_id)
except Exception as ex:
    print(ex.message)
finally:
    face_client.person_group.create(group_id, 'employees')
    print ('Group created!')

Una vez creado el grupo de personas, podemos agregar una persona para cada empleado que queramos incluir en el grupo y, después, agregar varias fotografías de cada persona para que Face pueda analizar las características faciales de cada persona. Lo ideal es que las imágenes muestren a la misma persona con diferentes poses y gestos.

Agregaremos un solo empleado llamado Wendell y tres fotografías suyas.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline

# Agregar una persona (Wendell) al grupo
wendell = face_client.person_group_person.create(group_id, 'Wendell')

# Obtener una foto de wendell
folder ='../data/face/wendell'
wendell_pics = os.listdir(folder)

# Registrar las fotos
i = 0
fig = plt.figure(figsize=(8, 8))
for pic in wendell_pics:
    # Agregue cada foto a la persona en el grupo de personas
    img_path = os.path.join(folder, pic)
    img_stream = open(img_path, "rb")
    face_client.person_group_person.add_face_from_stream(group_id, wendell.person_id, img_stream)

    # Muestre cada imagen
    img = Image.open(img_path)
    i +=1
    a=fig.add_subplot(1,len(wendell_pics), i)
    a.axis('off')
    imgplot = plt.imshow(img)
plt.show()

Una vez agregadas la persona y las fotografías, podemos entrenar a Face para que reconozca a Wendell.

In [ ]:
face_client.person_group.train(group_id)
print('Trained!')

Una vez entrenado el modelo, podemos usarlo para identificar caras reconocidas en una imagen.

In [ ]:
# Obtener los identificadores de caras en una segunda imagen
image_path ='../data/face/employees.jpg'
image_stream = open(image_path, "rb")
image_faces = face_client.face.detect_with_stream(image=image_stream)
image_face_ids = list(map(lambda face: face.face_id, image_faces))

# Obtener los nombres de caras reconocidas
face_names = {}
recognized_faces = face_client.face.identify(image_face_ids, group_id)
for face in recognized_faces:
    person_name = face_client.person_group_person.get(group_id, face.candidates[0].person_id).name
    face_names[face.face_id] = person_name

# Mostrar las caras reconocidas
faces.show_recognized_faces(image_path, image_faces, face_names)